In [6]:
import json
import requests
from datetime import datetime, timedelta
from transformers.agents import Tool
from unsloth import FastLanguageModel

In [2]:
class WeatherTool(Tool):
    name = "weather_api"
    description = "Retrieves weather information for a specific city."
    inputs = {
        "city": {
            "type": "string",
            "description": "The name of the city to get weather information for."
        },
        "forecast_type": {
            "type": "string",
            "description": "Either 'current' for current weather or 'forecast' for future weather."
        }
    }
    output_type = "text"

    def __init__(self, api_key):
        super().__init__()
        self.api_key = api_key

    def __call__(self, city: str, forecast_type: str = "current") -> str:
        base_url = "http://api.openweathermap.org/data/2.5"
        
        if forecast_type == "current":
            endpoint = f"{base_url}/weather"
        else:
            endpoint = f"{base_url}/forecast"
        
        params = {
            "q": city,
            "appid": self.api_key,
            "units": "metric"
        }
        
        response = requests.get(endpoint, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if forecast_type == "current":
                temp = data['main']['temp']
                description = data['weather'][0]['description']
                return f"The current weather in {city} is {description} with a temperature of {temp}°C."
            else:
                forecast = data['list'][0]  # Get the first forecast entry
                temp = forecast['main']['temp']
                description = forecast['weather'][0]['description']
                date_time = forecast['dt_txt']
                return f"The forecast for {city} on {date_time} is {description} with a temperature of {temp}°C."
        else:
            return f"Error: Unable to fetch weather data for {city}. Status code: {response.status_code}"


In [4]:
from transformers import Tool, load_tool, ReactCodeAgent, HfEngine

In [5]:
# Initialize the WeatherTool
api_key = 'c6dfc4d92a8f972d237ef696ec87b37a'
weather_tool = WeatherTool(api_key)

# Initialize the LLM engine
llm_engine = HfEngine("unsloth/Phi-3-mini-4k-instruct") 

# Initialize the agent with the weather tool
agent = ReactCodeAgent(tools=[weather_tool], llm_engine=llm_engine)

# Run it!
result = agent.run(
    "What's the weather like in London today?",
)
print(result)

======== New task ========
What's the weather like in London today?
Error in generating llm output: Model not loaded on the server: https://api-inference.huggingface.co/models/unsloth/Phi-3-mini-4k-instruct/v1/chat/completions. Please retry with a higher timeout (current: 120)..
Traceback (most recent call last):
  File "/home/vatsal-patel/anaconda3/envs/unsloth_env/lib/python3.10/site-packages/huggingface_hub/utils/_errors.py", line 304, in hf_raise_for_status
    response.raise_for_status()
  File "/home/vatsal-patel/anaconda3/envs/unsloth_env/lib/python3.10/site-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 503 Server Error: Service Unavailable for url: https://api-inference.huggingface.co/models/unsloth/Phi-3-mini-4k-instruct/v1/chat/completions

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/vatsal-patel/anaconda3/

The current weather in London is a clear sky with a temperature of approximately 1.14°C (33.85°F). Considering these temperatures, it implies that it is quite cold and possibly the time is during the early morning or late night. It would be advisable to dress warmly if you plan to be outdoors.
